In [14]:
!pip install -q torchgeo==0.5.1 segmentation-models-pytorch==0.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.5 MB/s eta 0:00:00


In [16]:
# ==============================================================================
#           AI-Powered Flood Mapping (Final Code for Colab)
# ==============================================================================
# This script will now work correctly with the specified library versions.
# ==============================================================================

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import segmentation_models_pytorch as smp
from torchgeo.datasets import Sen1Floods11 # This import now works
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Configuration & Hyperparameters ---
DATA_ROOT = "data"
NUM_EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.001
# In Colab, num_workers is often best set to 2
NUM_WORKERS = 2

# --- 2. Model, Loss Function, and Optimizer Definition ---
# This automatically selects the Colab GPU if it's enabled
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Define the U-Net model using a pre-trained ResNet-34 encoder
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=2, # Sentinel-1 has 2 bands (VV, VH)
    classes=1,
).to(device)

# Use the correct loss function for binary segmentation
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- 3. Data Loading and Preparation ---
print("Downloading and preparing the dataset (this may take a few minutes)...")
# torchgeo will download the data to the temporary Colab storage
dataset = Sen1Floods11(root=DATA_ROOT, bands=('s1'), splits=["train", "val"], download=True)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(f"✅ Dataset prepared: {len(train_dataset)} training samples, {len(val_dataset)} validation samples.")

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# --- 4. Training Loop ---
print("\n🚀 Starting model training...")
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    for i, sample in enumerate(train_loader):
        images = sample['image'].to(device)
        masks = sample['mask'].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Training Loss: {epoch_loss:.4f}")

print("✅ Training finished.")

# --- 5. Evaluation and Visualization ---
print("\nVisualizing model predictions...")
model.eval()

# Get a random sample from the validation set to test on
sample_idx = np.random.randint(0, len(val_dataset))
sample = val_dataset[sample_idx]

image = sample['image'].unsqueeze(0).to(device)
true_mask = sample['mask'].squeeze().numpy()

with torch.no_grad():
    pred_mask_logits = model(image)
    pred_mask = torch.sigmoid(pred_mask_logits).squeeze().cpu().numpy() > 0.5

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.imshow(image.cpu().numpy()[0, 0, :, :], cmap='gray')
ax1.set_title("Input SAR Image (VV band)")
ax1.axis('off')

ax2.imshow(true_mask, cmap='gray')
ax2.set_title("Ground Truth Mask")
ax2.axis('off')

ax3.imshow(pred_mask, cmap='gray')
ax3.set_title("Model's Predicted Mask")
ax3.axis('off')

plt.tight_layout()
plt.savefig("prediction_result.png")
print("\n✅ Saved prediction result to prediction_result.png in the Colab file browser.")
plt.show()

ImportError: cannot import name 'Sen1Floods11' from 'torchgeo.datasets' (/usr/local/lib/python3.12/dist-packages/torchgeo/datasets/__init__.py)